<a href="https://colab.research.google.com/github/boyila/124ofai_alphaml_sep4_2023/blob/main/Selenium_Web_Scraping_of_Glassdoor_Site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from itertools import repeat
import pandas as pd
import configparser
import numpy as np

# Setting up Webdriver and credentials

In [ ]:
# Initialize the WebDriver using Chrome
driver = webdriver.Chrome()
driver.maximize_window()  # Maximize the browser window

# Navigate to the Glassdoor website
driver.get("https://www.glassdoor.com/index.htm")

# Create a configparser object and read the configuration file
config = configparser.ConfigParser()
config.read("config.ini")

# Access the username and password from the configuration file
username = config["credentials"]["username"]
password = config["credentials"]["password"]

# Find and fill in the username field, then press Enter
uname = driver.find_element(By.XPATH, ".//*[@id='inlineUserEmail']")
uname.send_keys(username)
uname.send_keys(Keys.ENTER)

# Wait for the password field to be present, then fill it in and submit the form
upassword = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, ".//*[@id='inlineUserPassword']"))
)
upassword.send_keys(password)
upassword.submit()

SessionNotCreatedException: ignored

# Sales Manager Single-Page Step-by-Step

In [ ]:
# Navigate to the page with interview questions for sales managers
driver.get(
    "https://www.glassdoor.com/Interview/sales-manager-interview-questions-SRCH_KO0,13.htm"
)

# Initialize an empty list to store interview questions
question_list = []

# Find interview questions suing XPath looking for <h3 class='css-1jvs3tk edupdmz3'>
questions = driver.find_elements(By.XPATH, "//h3[@class='css-1jvs3tk edupdmz3']")

# Append the questions to the question_list
for question in questions:
    question_list.append(question.text)

print(question_list)

NameError: ignored

In [ ]:
# Find the number of responses looking for <a class='mt-lg d-flex align-items-center'>
response_elements = driver.find_elements(
    By.XPATH, "//a[@class='mt-lg d-flex align-items-center']"
)
# Initialize empty list
response_number = []

# Click on elements to load more responses, if any
for element in response_elements[1:]:
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of(element))
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", element)

# Get the text from the response elements and convert to an integer
for element in response_elements:
    response_number.append(element.text)

number_responses = [int(text[0]) for text in response_number]
print(number_responses)

[4, 1, 1, 1, 1, 1, 8, 7, 7, 8]


In [ ]:
# Create a new question list repeating each question based on the number of responses
new_question_list = []
i = 0
for number in number_responses:
    new_question_list.extend(repeat(question_list[i], number))
    i += 1

# Find and click on elements to load more responses, if any, finding <a class='col d-flex justify-content-center py mb'>
show_more_elements = driver.find_elements(
    By.XPATH, "//a[@class='col d-flex justify-content-center py mb']"
)

try:
    for element in show_more_elements:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(element))
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
except TimeoutException as ex:
    print(str(ex))
    pass

# Clicking on 'More' to reveal additional content
more_elements = driver.find_elements(By.LINK_TEXT, "More")

for element in more_elements:
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of(element))
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", element)

# Find and extract the answers
answers = driver.find_elements(By.XPATH, "//p[@class='css-1jbawm2 edupdmz9']")

answer_list = []

# Append the answers to the answer_list
for answer in answers:
    answer_list.append(answer.text)

print(answer_list)

['Ask what they see in a pen. If he remembers the last pen he used, and what price he would put on a pen he uses for signing big important contracts. As soon as he names a price... sold Less', "this pen has an option for lifetime ink cartridge's. So if you buy the bundle with lifetime cartridges You will never need another pen again. I f you buy 50 pens the 1st pen is at no charge. these would be great for advertising purposes. We can engrave each pen with your company information on it. this promotion ends this week. How many would you like? Less", 'Me: How can I sell you something you already own? VP: It\'s your pen now - sell it to me. Me: My pen? Right?" VP: Yes, sell it to me. Me: You\'re sure about this being my pen? VP: That\'s what I said. Me: It\'s not for sale. VP: What do you mean? Me: I\'ll get a better price from someone who wants the pen. Trying to sell a product to a person who is giving the same item away for free is like trying to sell sand at the beach. VP: OK, I get 

In [ ]:
# Create a DataFrame from the questions and answers
df = pd.DataFrame(
    list(zip(new_question_list, answer_list)), columns=["Question", "Answer"]
)

# Iterating through Additional Pages

In [ ]:
# Initialize list to append completed dataframes from each page
df_list = []

# Initialize the WebDriver
for integer in range(2, 21):
    # Construct the URL based on the integer
    url = f"https://www.glassdoor.com/Interview/sales-manager-interview-questions-SRCH_KO0,13_IP{str(integer)}.htm"

    # Open the URL in the WebDriver
    driver.get(url)

    question_list = []
    # Find interview questions using XPath
    questions = driver.find_elements(By.XPATH, "//h3[@class='css-1jvs3tk edupdmz3']")

    # Iterate through the questions and append them to the question_list
    for question in questions:
        question_list.append(question.text)

    # Find the number of responses
    response_elements = driver.find_elements(
        By.XPATH, "//a[@class='mt-lg d-flex align-items-center']"
    )
    response_number = []

    # Print out the number of responses
    for element in response_elements[1:]:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(element))
        time.sleep(1)
        driver.execute_script("arguments[0].click();", element)

    for element in response_elements:
        response_number.append(element.text)

    # Extract the number of responses as integers
    number_responses = [int(text[0]) for text in response_number]
    print(number_responses)

    new_question_list = []
    num = 0
    for number in number_responses:
        # Repeat each question according to the number of responses
        new_question_list.extend(repeat(question_list[num], number))
        num += 1

    show_more_elements = driver.find_elements(
        By.XPATH, "//a[@class='col d-flex justify-content-center py mb']"
    )

    try:
        for element in show_more_elements:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.visibility_of(element))
            time.sleep(1)
            driver.execute_script("arguments[0].click();", element)
    except TimeoutException as ex:
        print(str(ex))
        pass

    # Clicking on 'More' to expose the remainder of the answer statement or prompt by looking a link text of 'More' (these elements only have <a link=''>)
    more_elements = driver.find_elements(By.LINK_TEXT, "More")

    # Print out the number of responses
    for element in more_elements:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(element))
        time.sleep(1)
        driver.execute_script("arguments[0].click();", element)

    # Find the answers
    answers = driver.find_elements(By.XPATH, "//p[@class='css-1jbawm2 edupdmz9']")

    answer_list = []

    # Iterate through the answers and append them to the answer_list
    for answer in answers:
        answer_list.append(answer.text)

    # Check if the lengths of new_question_list and answer_list are the same
    if len(new_question_list) == len(answer_list):
        # Create a DataFrame from the questions and answers and append it to df_list
        df = pd.DataFrame(
            list(zip(new_question_list, answer_list)), columns=["Question", "Answer"]
        )
        df_list.append(df)
    elif len(new_question_list) < len(answer_list):
        diff = len(answer_list) - len(new_question_list)
        new_question_list.extend(diff * np.nan)
        print('Add to new_question_list:', diff)
    else:
        diff = len(new_question_list) - len(answer_list)
        answer_list.extend(diff * np.nan)
        print('Add to answer_list:', diff)


    # Wait for 10 seconds before proceeding to the next iteration
    time.sleep(10)

[6, 6, 6, 7, 5, 5, 5, 5, 3, 4]
[4, 4, 4, 4, 4, 3, 4, 6, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 2, 3, 3]
[3, 2, 3, 2, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 1, 3, 3, 3, 3, 3, 3]
[3, 2, 1, 4, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 2, 3, 3, 3, 1, 2, 2, 2]
[2, 3, 2, 2, 1, 2, 2, 2, 2, 2]
[2, 2, 1, 1, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 1, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 1, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 1, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 1, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


In [ ]:
# Perform row-wise concatenation to join all the dataframes together
entire_df = pd.concat(df_list, axis=0)

# Perform row-wise concentation on the original and newly formed dataframe
total_df = pd.concat([df, entire_df], axis=0)
total_df = total_df.reset_index()
total_df.head(20)

,index,Question,Answer
0,0,What do you know about Uline?,What do you know about Uline? A family-owned b...
1,1,What do you know about Uline?,Uline is a family-owned distributor of shippin...
2,2,Why do you want to be in logistics?,Warehouse executive
3,3,Why do you want to be in logistics?,I love the challenge on a daily basis.
4,4,What are the key components of a successful sa...,"Be aggressive, outgoing and organized."
5,5,What are the key components of a successful sa...,Someone that knows how to discover and uncover...
6,6,Can you read people?... What do I think of you?,I responded as of course being a sales person....
7,7,Can you read people?... What do I think of you?,"Answer is! I like this guy, or girl because yo..."
8,8,Why do you think you'll be successful in this ...,"When I commit to something, I see it through. ..."
9,9,Why do you think you'll be successful in this ...,Im very flexible in working hour. Im keen to d...


In [ ]:
total_df.to_csv("sales_manager_interview_qa.csv")
len(total_df)

499